In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import BatchNormalization

es = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

EPOCHS = 1000

In [2]:
df = pd.read_csv('all_merged.csv', sep=",")

In [3]:
df

,Unnamed: 0,title,review,rating
0,0,"Occasionally awkward, but very good overall",Rika Nishina (Megumi Okina) works for a social...,9/10
1,1,"Non-Linear, yet Effective and Creepy.","Although the word ""grudge"" doesn't quite fit t...",10/10
2,2,Pretty Good,"""Ju-On, The Grudge"" is not an easy movie to fi...",7/10
3,3,Very Scary Japanese Horror Movie,"In Japan, when the volunteer social assistant ...",8/10
4,4,A genuinely creepy Japanese ghost story master...,Following in the immensely successful footstep...,10/10
...,...,...,...,...
4782,413,Japanese Horror Remakes... from this author it...,"Now having seen both The Ring and Dark Water, ...",4/10
4783,414,I might have gotten startled ONCE during this ...,"THe movie is slow, grinding, and uneventful. A...",4/10
4784,415,That Movie Blew Harder than Yeltsin into a Bre...,I've never written comments on a movie before....,3/10
4785,416,"The ""Dark Water"" is really dark, for a reason.","I'm 47, married 26 years, four kids. I was rai...",1/10


In [4]:
df = df.drop('Unnamed: 0', axis=1)

In [5]:
df

,title,review,rating
0,"Occasionally awkward, but very good overall",Rika Nishina (Megumi Okina) works for a social...,9/10
1,"Non-Linear, yet Effective and Creepy.","Although the word ""grudge"" doesn't quite fit t...",10/10
2,Pretty Good,"""Ju-On, The Grudge"" is not an easy movie to fi...",7/10
3,Very Scary Japanese Horror Movie,"In Japan, when the volunteer social assistant ...",8/10
4,A genuinely creepy Japanese ghost story master...,Following in the immensely successful footstep...,10/10
...,...,...,...
4782,Japanese Horror Remakes... from this author it...,"Now having seen both The Ring and Dark Water, ...",4/10
4783,I might have gotten startled ONCE during this ...,"THe movie is slow, grinding, and uneventful. A...",4/10
4784,That Movie Blew Harder than Yeltsin into a Bre...,I've never written comments on a movie before....,3/10
4785,"The ""Dark Water"" is really dark, for a reason.","I'm 47, married 26 years, four kids. I was rai...",1/10


In [6]:
df['review'][88]

'The "spirit" in this movie was kind of creepy, but I felt like I wasted an afternoon watching this piece of pooh. It didn\'t tie itself together very well, the acting was second rate and it was very slow. I thought there was a lot of potential with this, but they way it was done barely kept me awake. It would be very interesting to see someone take this story to another level. I\'m not sure what else to say about this, but I want my post to make it online now that I have invested the time to review this lame movie. Sorry to be dragging on... but 10 lines of text are required. We\'re almost there... Hang on... Wow! I feel like I\'m creating a movie like the Grudge, now. Long, dull and a waste of time to deal with. Oh well, we\'re done now!'

In [7]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from tqdm import tqdm
from nltk import sent_tokenize,word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [8]:
main = []

# Storing all punctuations using RE library like !;,"% etc
re_puncs = re.compile('[%s]' % re.escape(string.punctuation))
# Storing all stop words like a, an, the, when, there, this etc
stop_word  = set(stopwords.words('english'))
stop_word.add("im")
# print(stop_word)


In [9]:
sentences_listed = [line.split(" ") for line in df['review']]

import gensim

word_model = gensim.models.Word2Vec(sentences = sentences_listed, vector_size = 100)
words = list(word_model.wv.key_to_index)
print(len(words))

11398


In [10]:
!pip install gensim

In [11]:
word_model.wv.most_similar("actress")

[('role', 0.9090754389762878),
 ('performance', 0.904512882232666),
 ('Connelly', 0.8993149399757385),
 ('actress,', 0.88432377576828),
 ('Watts', 0.8794669508934021),
 ('Jennifer', 0.8657869696617126),
 ('beautiful', 0.8446453213691711),
 ('Browning', 0.839723527431488),
 ('journalist.', 0.8318081498146057),
 ('Naomi', 0.8311673402786255)]

In [12]:
word_model.wv.most_similar("death")

[('living', 0.9084473848342896),
 ('visions', 0.8978703022003174),
 ('spirit', 0.8877584934234619),
 ('father', 0.8806615471839905),
 ('victim', 0.8788247108459473),
 ('family', 0.8764746785163879),
 ('niece', 0.8754598498344421),
 ('body', 0.872402548789978),
 ('curse', 0.8700237274169922),
 ('murdered', 0.8697793483734131)]

In [13]:
word_model.wv.most_similar("scary")

[('bad', 0.8252667784690857),
 ('good', 0.762544572353363),
 ('boring', 0.7020155787467957),
 ('scary,', 0.6929928064346313),
 ('scary.', 0.6781390905380249),
 ('frightening', 0.6706538796424866),
 ('fun', 0.6651389002799988),
 ('stupid', 0.6647598147392273),
 ('truly', 0.6292786598205566),
 ('bad,', 0.62757408618927)]

In [14]:
# Making Lemmatizing object
lem = WordNetLemmatizer()
# Using Porter Stemmer
p_stem = PorterStemmer()

# Traversing whole dataset
for i in tqdm(range(len(df['review']))):
    # Tokenization
    tokens = word_tokenize(str(df['review'][i]))
    # Converting all characters to lower case
    tokens = [w.lower() for w in tokens]
    # Remove all punctuations from sentenses
    tokens = [re_puncs.sub('', w) for w in tokens]
    # Checking all words is alphabets or not
    tokens = [i for i in tokens if i.isalpha()]
    # Removing all stop words from the sentenses
    tokens = [w for w in tokens if w not in stop_word]
    # Doing Lemmatizing of words
    tokens = [lem.lemmatize(w) for w in tokens]
    # Stemming process
    tokens = [p_stem.stem(w) for w in tokens]
    # Finally convert to string
    r = ' '.join(tokens)
    # Storing the final string into main list
    main.append(r)


100%|██████████| 4787/4787 [00:39<00:00, 120.27it/s]


In [15]:
df['translated'] = main

In [16]:
df

,title,review,rating,translated
0,"Occasionally awkward, but very good overall",Rika Nishina (Megumi Okina) works for a social...,9/10,rika nishina megumi okina work social servic a...
1,"Non-Linear, yet Effective and Creepy.","Although the word ""grudge"" doesn't quite fit t...",10/10,although word grudg nt quit fit bill part titl...
2,Pretty Good,"""Ju-On, The Grudge"" is not an easy movie to fi...",7/10,juon grudg easi movi find america least nt fir...
3,Very Scary Japanese Horror Movie,"In Japan, when the volunteer social assistant ...",8/10,japan volunt social assist rika nishina megumi...
4,A genuinely creepy Japanese ghost story master...,Following in the immensely successful footstep...,10/10,follow immens success footstep ring juon grudg...
...,...,...,...,...
4782,Japanese Horror Remakes... from this author it...,"Now having seen both The Ring and Dark Water, ...",4/10,seen ring dark water begin wonder director com...
4783,I might have gotten startled ONCE during this ...,"THe movie is slow, grinding, and uneventful. A...",4/10,movi slow grind unev first minut see first sec...
4784,That Movie Blew Harder than Yeltsin into a Bre...,I've never written comments on a movie before....,3/10,never written comment movi never felt rob chea...
4785,"The ""Dark Water"" is really dark, for a reason.","I'm 47, married 26 years, four kids. I was rai...",1/10,marri year four kid rais watch televis movi sa...


In [17]:
#df['translated'][88]

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['translated'])

In [19]:
print(len(tokenizer.word_counts))

22923


In [20]:
counter = 0
for key, value in tokenizer.word_counts.items():
    if value > 2400:
        print(key, value)
        counter+=1

print(counter)

one 4912
film 9819
realli 3083
nt 8349
watch 3920
good 3593
get 2556
stori 3056
end 2875
like 4527
see 3767
horror 6036
well 2449
make 3008
movi 13409
time 2782
scare 2522
scari 2650
ring 3209
go 2819
origin 2743
21


**grupa 1** fright(ened), shocked/shocking, horror/horrified, horrendous, overwhelming, terror/terrified, disturbing, bloody, disgust(ing), brutal, gory/gore, barbarous, repulsed/-ing, gross(ed out)

In [21]:
count1 = 0
num=tokenizer.word_counts['fright']
print(num)
count1+=num

118


In [22]:
num=tokenizer.word_counts['frighten']
print(num)
count1+=num

518


In [23]:
num=tokenizer.word_counts['shock']
print(num)
count1+=num

438


In [24]:
num=tokenizer.word_counts['horrifi']
print(num)
count1+=num

140


In [25]:
num=tokenizer.word_counts['horrif']
print(num)
count1+=num

94


In [26]:
num=tokenizer.word_counts['horrend']
print(num)
count1+=num

15


In [27]:
num=tokenizer.word_counts['overwhelm']
print(num)
count1+=num

24


In [28]:
num=tokenizer.word_counts['overwhelming']
print(num)
count1+=num

1


In [29]:
num=tokenizer.word_counts['disturb']
print(num)
count1+=num

453


In [30]:
num=tokenizer.word_counts['terror']
print(num)
count1+=num

267


In [31]:
num=tokenizer.word_counts['blood']
print(num)
count1+=num

262


In [32]:
num=tokenizer.word_counts['terrifi']
print(num)
count1+=num

421


In [33]:
num=tokenizer.word_counts['disgust']
print(num)
count1+=num

34


In [34]:
num=tokenizer.word_counts['brutal']
print(num)
count1+=num

34


In [35]:
num=tokenizer.word_counts['gore']
print(num)
count1+=num

511


In [36]:
num=tokenizer.word_counts['repuls']
print(num)
count1+=num

17


In [37]:
num=tokenizer.word_counts['gross']
print(num)
count1+=num

32


In [38]:
print('WYNIK DLA GRUPY PIERWSZEJ TOOOOO',count1)

WYNIK DLA GRUPY PIERWSZEJ TOOOOO 3379


**grupa 2** afraid, alarmed, threatening, hair-rising, distress, helpless(ness), anxiety/anxious, worried, apprehension, uneasy, chilling, eerie, intimidating, unnerving, appalling

In [43]:
count2=0
num=tokenizer.word_counts['distress']
print(num)
count2+=num

18


In [44]:
num=tokenizer.word_counts['afraid']
print(num)
count2+=num

129


In [45]:
num=tokenizer.word_counts['alarm']
print(num)
count2+=num

11


In [46]:
num=tokenizer.word_counts['threaten']
print(num)
count2+=num

47


In [47]:
num=tokenizer.word_counts['anxious']
print(num)
count2+=num

2


In [48]:
num=tokenizer.word_counts['anxieti']
print(num)
count2+=num

27


In [49]:
num=tokenizer.word_counts['worri']
print(num)
count2+=num

61


In [50]:
num=tokenizer.word_counts['apprehens']
print(num)
count2+=num

3


In [51]:
num=tokenizer.word_counts['uneasi']
print(num)
count2+=num

54


In [52]:
num=tokenizer.word_counts['chilli']
print(num)
count2+=num

5


In [53]:
num=tokenizer.word_counts['eeri']
print(num)
count2+=num

276


In [54]:
num=tokenizer.word_counts['helpless']
print(num)
count2+=num

19


In [55]:
num=tokenizer.word_counts['unnerv']
print(num)
count2+=num

83


In [56]:
num=tokenizer.word_counts['appal']
print(num)
count2+=num

8


In [57]:
num=tokenizer.word_counts['fear']
print(num)
count2+=num

468


In [58]:
print('WYNIK DLA GRUPY DRUGIEJ TOOOOO',count2)

WYNIK DLA GRUPY DRUGIEJ TOOOOO 1211
